In [1]:
import pickle
import numpy as np
import pandas as pd
import json

In [2]:
with open('Run11_list.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [3]:
# Define the range for Tstamp_us
lower_bound = 1650 * ( 60 * 1e6)
upper_bound = 1700 * ( 60 * 1e6)

# Filter the DataFrame
data = data[(data['Tstamp_us'] >= lower_bound) & (data['Tstamp_us'] <= upper_bound)]

# Reset index if needed
data.reset_index(drop=True, inplace=True)

In [4]:
print(data)

         Brd  Ch  LG   HG     Tstamp_us  event_ids   xpos   ypos      zpos
0          0   0  67  160  9.900006e+10     112751  50.01  18.37   24.0526
1          0   1  47  196  9.900006e+10     112751  77.64  34.32   24.0526
2          0   2  50  177  9.900006e+10     112751  22.37  34.32   24.0526
3          0   3  33  166  9.900006e+10     112751  50.01  50.28   24.0526
4          0   4  40  175  9.900006e+10     112751  77.64  66.23   24.0526
...      ...  ..  ..  ...           ...        ...    ...    ...       ...
2290555    2  59  17  100  1.019964e+11     153880  26.10 -26.36  241.2734
2290556    2  60   0   61  1.019964e+11     153880 -73.90  74.16  268.4260
2290557    2  61  32   61  1.019964e+11     153880 -26.10  74.16  268.4260
2290558    2  62  24    0  1.019964e+11     153880 -73.90  26.36  268.4260
2290559    2  63   0   24  1.019964e+11     153880 -26.10  26.36  268.4260

[2290560 rows x 9 columns]


In [5]:
LG = np.array(data["LG"])
event_ids = np.array(data["event_ids"])
xpos = np.array(data["xpos"])
ypos = np.array(data["ypos"])
zpos = np.array(data["zpos"])

#LG = LG - 50.39

mask = LG > 0

LG = LG[mask]
event_ids = event_ids[mask]
xpos = xpos[mask]
ypos = ypos[mask]
zpos = zpos[mask]

events = list(set(event_ids))

In [14]:
use_log_scale = False  # Set to True for logarithmic scale, False for linear scale
time_window = 1 # All hits below this time value will be given artificial hit times

data = []

numEvts = 50
if len(events) < 20:
    numEvts = len(x)
    


for i in events[0:numEvts]:
    event_mask_1 = False
    event_mask_2 = False
    
    event = {
        "infoText": "CALI BNL Prototype Event #" + str(i),
        "colorScale": {}
    }
    
    mask = (event_ids == i)

    # Filter out energies less than or equal to 0
    valid_energies = [energy for energy in LG[mask] if energy > 300]

    if valid_energies:
        event_mask_1 = True
        min_energy = min(valid_energies)
        max_energy = max(valid_energies)

        event["colorScale"]["min"] = float(min_energy)
        event["colorScale"]["max"] = float(max_energy)

        if use_log_scale:
            # Logarithmic scaling
            normalized_energy = [np.log(energy / min_energy) / np.log(max_energy / min_energy) if energy > 0 else energy for energy in LG[mask]]
        else:
            # Linear scaling
            normalized_energy = [(energy - min_energy) / (max_energy - min_energy) if energy > 0 else energy for energy in LG[mask]]
    else:
        # If all energies are 0 or less, retain the original values
        normalized_energy = LG[mask]

    objects = []

    for j in range(len(LG[mask])):
        dist = np.sqrt((xpos[mask][j] / 200) ** 2 + (ypos[mask][j] / 200) ** 2 + (zpos[mask][j] / 200) ** 2)
        c = 0.299792
        time = dist / c
        if normalized_energy[j] > 0:
            event_mask_2 = True
            obj = {
                "type": "hit",
                "time": float(time),
                "x": float(xpos[mask][j] / 200),
                "y": float(ypos[mask][j] / 200),
                "z": float(zpos[mask][j] / 200),
                "color": float(normalized_energy[j])
            }

            objects.append(obj)
    if event_mask_1 & event_mask_2:
        event["objects"] = objects
        data.append(event)

eventPath = 'Events.json'
# Write data to JSON file
with open(eventPath, 'w') as json_file:
    json.dump(data, json_file, indent=2)

/Users/spreins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in long_scalars
/Users/spreins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in long_scalars


In [8]:
import json

template_parameters = {
    "sides": 4,
    "rmin1": 0,
    "rmax1": 1.386/2,
    "rmin2": 0,
    "rmax2": 1.386/2,
    "lengthOut": 0.271526/2,
    "lengthIn": 0.271526/2,
    "offsetIn": 0,
    "angle": 0,
    "R": 0.5882352941176471,
    "G": 0.5882352941176471,
    "B": 1.0
}

# Extend the data for additional layers
for i in range(1, 18): 
    new_entry = {
        "name": f"Layer{i}",
        "parameters": template_parameters.copy()  # Create a copy of the template parameters
    }
    new_entry["parameters"]["offset"] = (0.1358/2) + (0.271526/2) * (i - 1)  # Update the offset
    data.append(new_entry)


# Write the updated data to a new JSON file
with open('Detector.json', 'w') as file:
    json.dump(data, file, indent=2)  # Save the updated data to a new JSON file